In [0]:
%pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month, year, avg, to_date
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName("Evapotranspiration_MLlib").getOrCreate()

df = spark.read.option("header", "true").option("inferSchema", "true").csv("/opt/data/weatherData.csv")

print("Data loaded successfully!")
print(f"Total records: {df.count()}")
# df.printSchema()

In [ ]:
%pyspark
df_clean = df.withColumnRenamed("precipitation_hours (h)", "precipitation_hours").withColumnRenamed("sunshine_duration (s)", "sunshine_duration") \
            .withColumnRenamed("wind_speed_10m_max (km/h)", "wind_speed").withColumnRenamed("et0_fao_evapotranspiration (mm)", "evapotranspiration")

df_clean = df_clean.withColumn("date_parsed", to_date(col("date"), "M/d/yyyy")).withColumn("year", year("date_parsed")).withColumn("month", month("date_parsed"))

# Filter for May (month = 5)
df_may = df_clean.filter(col("month") == 5)

df_features = df_may.select("precipitation_hours", "sunshine_duration", "wind_speed", "evapotranspiration").na.drop()

print(f"May records: {df_features.count()}")
df_features.describe().show()


In [ ]:
%pyspark
feature_cols = ["precipitation_hours", "sunshine_duration", "wind_speed"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_raw")
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withStd=True, withMean=True)

df_assembled = assembler.transform(df_features)
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

print("Features assembled and scaled!")
df_scaled.select("features", "evapotranspiration").show(5, truncate=False)

In [ ]:
%pyspark
# Split data: 80% training, 20% validation
train_data, test_data = df_scaled.randomSplit([0.8, 0.2], seed=42)

print(f"Training samples: {train_data.count()}")
print(f"Validation samples: {test_data.count()}")

In [ ]:
%pyspark
# Initialize Linear Regression
lr = LinearRegression(
    featuresCol="features",
    labelCol="evapotranspiration",
    maxIter=100,
    regParam=0.3,
    elasticNetParam=0.8)

lr_model = lr.fit(train_data)

print("=== Linear Regression Model ===")
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")
print(f"R-squared (training): {lr_model.summary.r2}")

In [ ]:
%pyspark
# Random Forest for comparison
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="evapotranspiration",
    numTrees=100,
    maxDepth=10,
    seed=42)

rf_model = rf.fit(train_data)

print("=== Random Forest Model ===")
print(f"Feature Importances: {rf_model.featureImportances}")
print(f"Features: {feature_cols}")

from pyspark.ml.regression import GBTRegressor # Remove this

# Gradient Boosted Trees (Adding for better non-linear performance)
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="evapotranspiration",
    maxIter=100,
    seed=42)

gbt_model = gbt.fit(train_data)
print("\n=== Gradient Boosted Trees (GBT) Model ===")
print(f"Feature Importances: {gbt_model.featureImportances}")
print(f"Features: {feature_cols}")

In [ ]:
%pyspark
# Model evaluation
evaluator_rmse = RegressionEvaluator(labelCol="evapotranspiration", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="evapotranspiration", predictionCol="prediction", metricName="r2")
evaluator_mae = RegressionEvaluator(labelCol="evapotranspiration", predictionCol="prediction", metricName="mae")

# Linear Regression predictions
lr_predictions = lr_model.transform(test_data)
lr_rmse = evaluator_rmse.evaluate(lr_predictions)
lr_r2 = evaluator_r2.evaluate(lr_predictions)

# Random Forest predictions
rf_predictions = rf_model.transform(test_data)
rf_rmse = evaluator_rmse.evaluate(rf_predictions)
rf_r2 = evaluator_r2.evaluate(rf_predictions)

# GBT predictions
gbt_predictions = gbt_model.transform(test_data)
gbt_rmse = evaluator_rmse.evaluate(gbt_predictions)
gbt_r2 = evaluator_r2.evaluate(gbt_predictions)
gbt_mae = evaluator_mae.evaluate(gbt_predictions)

print("=== Linear Regression Evaluation ===")
print(f"RMSE: {lr_rmse:.4f}, R2: {lr_r2:.4f}")

print("\n=== Random Forest Evaluation ===")
print(f"RMSE: {rf_rmse:.4f}, R2: {rf_r2:.4f}")

print("\n=== Gradient Boosted Trees Evaluation ===")
print(f"RMSE: {gbt_rmse:.4f}")
print(f"R-2: {gbt_r2:.4f}")
print(f"MAE: {gbt_mae:.4f}")

In [ ]:
%pyspark
# Find patterns where evapotranspiration < 1.5mm
low_et = df_features.filter(col("evapotranspiration") < 1.5)

print("=== Conditions for Evapotranspiration < 1.5mm ===")
print(f"Number of days with ET < 1.5mm: {low_et.count()}")

low_et_stats = low_et.agg(
    avg("precipitation_hours").alias("mean_precipitation_hours"),
    avg("sunshine_duration").alias("mean_sunshine_duration"),
    avg("wind_speed").alias("mean_wind_speed"))

print("\nMean values when evapotranspiration < 1.5mm:")
low_et_stats.show()

In [ ]:
%pyspark
import random
from pyspark.sql.types import StructType, StructField, DoubleType
from pyspark.sql.functions import avg, col

print("PREDICTION: Mean Weather Conditions for Target ET < 1.5mm (GBT + Random Search)")
print("(Goal: Find average weather conditions that reliably result in < 1.5mm ET)")

# Random Grid Search Parameters
# Based on March analysis: High Rain, Low Sun, Variable Wind
NUM_SAMPLES = 50000
MIN_SUNSHINE = 0.0         # Allow 0 hours
MAX_SUNSHINE = 3600.0      # Cap at 1 hour (3600s) to force low-sunshine similarity
MIN_WIND = 15.0            # Range around the 18.5 km/h mean
MAX_WIND = 25.0            # Allow higher wind
MIN_PRECIP = 20.0          # Force high rain (centered on 22h)
MAX_PRECIP = 24.0          # Max possible

print(f"Generating {NUM_SAMPLES} synthetic realistic weather points...")

synthetic_data = []
for _ in range(NUM_SAMPLES):
    synthetic_data.append((
        random.uniform(MIN_PRECIP, MAX_PRECIP),
        random.uniform(MIN_SUNSHINE, MAX_SUNSHINE),
        random.uniform(MIN_WIND, MAX_WIND)
    ))

# Create DataFrame
schema = StructType([
    StructField("precipitation_hours", DoubleType(), True),
    StructField("sunshine_duration", DoubleType(), True),
    StructField("wind_speed", DoubleType(), True)
])

df_synthetic = spark.createDataFrame(synthetic_data, schema)

# Transform features
df_synthetic_assembled = assembler.transform(df_synthetic)
df_synthetic_scaled = scaler_model.transform(df_synthetic_assembled)

# Predict using GBT
predictions = gbt_model.transform(df_synthetic_scaled)

# --- Filter for Target and Calculate Means ---
# Keep ONLY the days where Predicted ET < 1.5
successful_days = predictions.filter(col("prediction") < 1.5)
success_count = successful_days.count()

print(f"\nSearch Complete.")
print(f"Found {success_count} days with ET < 1.5mm out of {NUM_SAMPLES} samples.")

if success_count > 0:
    # Calculate Mean of those successful days
    mean_values = successful_days.agg(
        avg("precipitation_hours").alias("avg_precip"),
        avg("sunshine_duration").alias("avg_sun"),
        avg("wind_speed").alias("avg_wind"),
        avg("prediction").alias("avg_et")
    ).collect()[0]
    
    recommended_stats = {
        'precip': mean_values["avg_precip"],
        'sun': mean_values["avg_sun"],
        'wind': mean_values["avg_wind"]
    }
    
    print(f"\n*** Recommended Mean Values for May 2026 (Target < 1.5mm) ***")
    print(f"Precipitation Hours: {recommended_stats['precip']:.2f} hours")
    print(f"Sunshine Duration: {recommended_stats['sun']:.2f} seconds ({recommended_stats['sun']/3600:.2f} hours)")
    print(f"Wind Speed: {recommended_stats['wind']:.2f} km/h")
    print(f"\nExpected Average ET with these conditions: {mean_values['avg_et']:.4f} mm")

else:
    print(f"\nFAILURE: Could not find any weather combinations with ET < 1.5mm.")
    print("Model indicates < 1.5mm is extremely difficult/impossible within these search parameters.")
